In [1]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time


In [ ]:
#Acesso ao banco de dados
def executar_conexao_bd():
    server = 'TBDBL-GLEISSON'
    database = 'Dados_Pavimento'
    username = 'gleisson.santos'
    password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

In [2]:
#Inicinado automação / abrindo navegador
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\gleisson.santos\.wdm\drivers\chromedriver\win32\111.0.5563.64]
C:\Users\GLEISS~1.SAN\AppData\Local\Temp/ipykernel_3896/3751352982.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
login = 't034183'
senha = 'CNB@2022'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [4]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(3)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'insum', 'GOSI_anchor')

In [ ]:
itens = ['INTDIG01 - LUVA PVC JS 40MM', 'INTDIG01 - JOELHO 90 PVC C/ANEL 100MM ESG PRED']

In [10]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').clear()
    
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt72-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-j_idt80-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt71-panel"]/input').click()

In [35]:
#Acessando e inserindo material
def campo_material(driver, material)
    campo1 = driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-tmpCodigoAutoComp-codAutoComp_input"]')
    campo1.click()
    campo1.clear()
    campo.send_keys(material)

In [36]:
#Acessando e inserindo quantidade e medir
def campo_quantidade(driver, qtd, medir)
    campo2 = driver.find_element(By.XPATH, '//*[@id="form-cpnlAssociarInsumo-quantidade-item"]')
    campo2.click()
    campo2.clear()
    campo2.send_keys(qtd)
    time.sleep(0.5)
    #Acessando e inserindo Disp p medir S/N
    select_element = driver.find_element(By.ID, 'form-cpnlAssociarInsumo-j_idt290-comboBooleanSN-item')
    select = Select(select_element)
    select.select_by_value(medir)
    # select.select_by_visible_text('Não')
    time.sleep(0.5)
    #Confirar insumo
    driver.find_element(By.XPATH,'//*[@id="form-cpnlAssociarInsumo-confirmar"]').click()

In [ ]:
    try:
        material = driver.find_element(By.CSS_SELECTOR, "label[title='Material Utilizado'] ").text
    except NoSuchElementException:
        material = 'None'
    
    try:
        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD'] ").text
    except NoSuchElementException:
        try:
            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD Instalado'] ").text
        except NoSuchElementException:
            try:
                hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro Instalado'] ").text
            except NoSuchElementException:
                try:
                    hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro'] ").text
                except NoSuchElementException:
                    try:
                        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Hidrômetro Instalado'] ").text
                    except NoSuchElementException:
                        hd = None

    return material, hd

In [ ]:
df = pd.read_sql('SELECT * FROM [- 700Com_material]', executar_conexao_bd())
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
#df = df.astype(str)

display(df)

In [ ]:
#Extrair valores e criar coluna Material insumado
dados = []
hds = []
for i, row in df.iterrows():
    a = preencher_ss(driver, row['SS'], row['OS'])
    time.sleep(0.3)
    dados.append(a[0])
    hds.append(a[1])
df['Material'] = dados
df['Hd'] = hds


In [ ]:
#Acessar tabela do BD e extrair apenas item do texto
m_encontrado = ['Mureta', 'mureta']

material = []

for i, row in df.iterrows():
    if any(x in row['Material'] for x in m_encontrado):
        material.append('Sim')
    else:
        material.append('Não')     
df['Mureta'] = material


df = df.astype(str)
df = pd.DataFrame(df, columns = ['SS', 'OS', 'Material', 'Mureta','Hd'])
#display(df)
mureta_df = df[df["Material"].str.contains("Mureta|mureta")]
display(mureta_df)

In [ ]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    comando_sql = "UPDATE [Insumar] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()